In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Import features
data1=pd.read_csv('../Results/PP_results/AAC_pp_cl.csv').iloc[:,2:-1].values
data2=pd.read_csv('../Results/PP_results/PAAC_pp_cl.csv').iloc[:,2:-1].values
data3=pd.read_csv('../Results/PP_results/APAAC_pp_cl.csv').iloc[:,2:-1].values
data4=pd.read_csv('../Results/PP_results/SOCN_pp_cl.csv').iloc[:,2:-1].values
data5=pd.read_csv('../Results/PP_results/GAAC_pp_cl.csv').iloc[:,2:-1].values
data6=pd.read_csv('../Results/PP_results/CKSAAGP_pp_cl.csv').iloc[:,2:-1].values
data7=pd.read_csv('../Results/PP_results/DPC_pp_cl.csv').iloc[:,2:-1].values
X1 = np.concatenate((data1, data2, data3, data4, data5, data6, data7), axis=1)

# Load the 70-dim GRU feature vectors
X2 = np.load("../features_70dim.npy")  # shape: (510, 70)
y = pd.read_csv('../Results/PP_results/AAC_pp_cl.csv').iloc[:,-1].values #Labels

#Concatenate the features
X=np.concatenate((X1,X2), axis=1)

In [3]:
print("Shape of the input features is as: ",  X.shape)
print("Total number of samples are as: ", len(X))

Shape of the input features is as:  (510, 140)
Total number of samples are as:  510


In [4]:
from sklearn.preprocessing import StandardScaler
#Standard Scaler
X_stand = np.array(X).reshape(-1, 140)
scaler = StandardScaler()
# X[:, 70:] = scaler.fit_transform(X[:, 70:])
X_stand = scaler.fit_transform(X_stand)

In [5]:
#Data preparation
from sklearn.model_selection import train_test_split
import torch

# Convert to PyTorch tensors
X_torch = torch.tensor(X_stand, dtype=torch.float32)
y_torch = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # shape: (510, 1)

#Use indices to separate the test dataset and the training dataset
n_samples=510
indices = np.arange(n_samples)


# X: features, y: labels
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X_torch, y_torch, indices, test_size=0.2, random_state=26, stratify=y_torch)

print("Number of samples in training set are as: ", len(X_train))
print("Number of samples in test set are as: ", len(X_test))
print("Shape of X_train is: ", X_train.shape)
print("Shape of X_test is: ", X_test.shape)
print("X_train or X_test is of: ", type(X_train))

Number of samples in training set are as:  408
Number of samples in test set are as:  102
Shape of X_train is:  torch.Size([408, 140])
Shape of X_test is:  torch.Size([102, 140])
X_train or X_test is of:  <class 'torch.Tensor'>


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import (
    recall_score, confusion_matrix, matthews_corrcoef, roc_auc_score
)

# Define the model class again (must match the original)
class FusionClassifier(nn.Module):
    def __init__(self, input_dim=140, hidden_dim=64, use_two_layers=True):
        super(FusionClassifier, self).__init__()
        
        self.use_two_layers = use_two_layers
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dropout = nn.Dropout(0.3)
        
        if self.use_two_layers:
            self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
            self.classifier = nn.Linear(hidden_dim // 2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        if self.use_two_layers:
            x = F.relu(self.fc2(x))
            x = self.dropout(x)
        logits = self.classifier(x)
        return logits

In [7]:
# Re-instantiate the model
model = FusionClassifier(input_dim=140, hidden_dim=64, use_two_layers=True)

# Load the saved state dict
model.load_state_dict(torch.load("linear_model_weights.pth", map_location=torch.device("cpu")))  # or "cuda" if GPU
model.eval()

FusionClassifier(
  (fc1): Linear(in_features=140, out_features=64, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (classifier): Linear(in_features=32, out_features=1, bias=True)
)

In [8]:
# Prediction section
with torch.no_grad():
    logits = model(X_test)
    probs = torch.sigmoid(logits)
    preds = (probs >= 0.5).int()

    accuracy = (preds == y_test.int()).float().mean().item()

    y_true = y_test.cpu().numpy().flatten()
    y_pred = preds.cpu().numpy().flatten()
    y_prob = probs.cpu().numpy().flatten()

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    recall = recall_score(y_true, y_pred)
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    mcc = matthews_corrcoef(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)

    print(f"Accuracy    : {accuracy:.4f}")
    print(f"Recall      : {recall:.4f}")
    print(f"Specificity : {specificity:.4f}")
    print(f"MCC         : {mcc:.4f}")
    print(f"AUC         : {auc:.4f}")

Accuracy    : 0.9412
Recall      : 0.9608
Specificity : 0.9216
MCC         : 0.8830
AUC         : 0.9904
